# 9. Basic Loss Function
本节旨在介绍[不同任务下基础的Loss函数](https://zhuanlan.zhihu.com/p/377799012)，并结合例子进行说明。

Loss Function 用于衡量模型输出与ground truth之间的误差，Loss越大意味着模型训练效果越差。

在训练模型时，我们通常采用gradient descent来最小化loss function，直到loss收敛。

此外，[PyTorch的其他损失函数](https://zhuanlan.zhihu.com/p/61379965)。

## 9.1 Regression Loss （回归损失）
回归损失通常针对连续型变量计算损失，常包含 **MAE (L1 Loss), MSE (L2 Loss)**。

### 9.1.1 Mean Absolute Loss (MAE) - L1 Loss
 MAE又称为L1 Loss，因为其与L1范数相似，都是计算样本间差的绝对值。

对于 模型预测值 $\hat{\bm{y}}=\{\hat{y}_i\}$ 与 样本真实值 $\bm{y}=\{y_i\}$，MAE计算其所有元素之差的绝对值，并返回其平均值：

$$MAE = \frac{1}{n}\sum_{i=1}^n|\hat{y}_i-y_i|$$

优势：稀疏性强，对离群点具有鲁棒性（因为离群点的梯度不会被MAE放大，使模型不会为了少数异常值而进行大范围的调整，牺牲其他正常样本）

缺陷：在原点处导数不连续，不同损失值的梯度一样大，使其梯度求解效率低下，导致收敛速度慢，不利于网络的学习

In [27]:
import torch
from torch import nn
from torch.nn import functional as F

output = torch.tensor([2, 3, 4], dtype = torch.float)
target = torch.tensor([3, 4, 5], dtype = torch.float)

# MAE Loss (Self-implemented)
def MAE_loss(output, target):
    diff = torch.abs(output - target)
    loss = torch.sum(diff)/diff.size(0)
    return loss

print(MAE_loss(output, target))

# Torch MAE Loss (L1 Loss)
loss = nn.L1Loss()
MAE = loss(output, target) # (|3-2|+|4-3|+|5-4|)/3 = 1
print(MAE)

tensor(1.)
tensor(1.)


### 9.1.2 Mean Squared Loss (MSE) - L2 Loss
MSE又称为L2 Loss，因为其与L2范数相似，都是计算样本间差的平方

对于 模型预测值 $\hat{\bm{y}}=\{\hat{y}_i\}$ 与 样本真实值 $\bm{y}=\{y_i\}$，MSE计算其所有元素之差的平方，并返回其平均值：

$$MSE = \frac{1}{n}\sum_{i=1}^n(\hat{y}_i-y_i)^2$$

优势：收敛速度快（梯度对大差值有放大效果），易于训练

缺陷：对异常值十分敏感，因为其梯度更新的方向容易受到离群点主导

In [28]:
# MSE Loss (Self-implemented)
def MSE_loss(output, target):
    diff = output-target
    diff = diff.pow(2)
    loss = torch.sum(diff)/diff.size(0)
    return loss

print(MSE_loss(output, target))

# Torch MSE Loss
loss = nn.MSELoss()
MSE = loss(output, target)
print(MSE)

tensor(1.)
tensor(1.)


## 9.2 Classification Loss（分类损失）
分类损失通常针对离散型变量计算损失，常包含KL散度损失（KL Divergence Loss）、交叉熵损失（Cross Entropy Loss）。

### 9.2.1 KL Divergence Loss
KL散度损失用于刻画两个概率分布间的差异程度，若两个分布差异较小，则KL散度趋向于0，否则KL散度将很大。KL散度定义为

<p align=center>
<img src="./fig/9-0.png" width=800>
</p>

通过KL散度可以衡量预测分类概率分布q（视为一个多项分布）与真实分类概率分布p间的差异，从而构建KL散度损失。

In [56]:
predicted = torch.tensor([5, 7, 2], dtype=torch.float)
predicted_softmax = F.softmax(predicted)
print(predicted_softmax)
ground_truth = torch.tensor([0.3, 0.5, 0.2], dtype=torch.float)

# KL Divergence Loss (Self divergence)
def KL_loss(predicted, ground_truth):
    tmp = torch.div(ground_truth, predicted)
    tmp = tmp.log()
    KL_loss = torch.sum(ground_truth.matmul(tmp))
    return KL_loss

print(KL_loss(predicted_softmax, ground_truth))

# Torch KL Div Loss
loss = nn.KLDivLoss(reduction="sum")
KLD_loss = loss(predicted_softmax.log(), ground_truth) # The input must be log(q)
print(KLD_loss)

tensor([0.1185, 0.8756, 0.0059])
tensor(0.7032)
tensor(0.7032)


/var/folders/fw/by0ngc4j67x7597xgdhn13xr0000gn/T/ipykernel_44932/3173961560.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predicted_softmax = F.softmax(predicted)



### 9.2.2 Cross Entropy Loss
交叉熵损失起源于信息论，从信息量->熵->交叉熵。

最早对信息量的定义为：若一件事情发生的概率很小，但其发生了，则该事件发生所包含的信息量将很大；若一件事情发生的概率很大，其发生了，则该事件发生所包含的信息量将很小。

<p align=center>
<img src="./fig/9-1.png" width=800>
</p>

如果将事件X的所有可能性列出来，就可以求得该事件信息量的期望（熵），因此熵定义为：

<p align=center>
<img src="./fig/9-2.png" width=700>
</p>

对于上一小节中阐述的用以衡量两个概率分布差异的KL散度（相对熵），将log拆开，可知真正衡量p与q分布间关系的为其中一项：

<p align=center>
<img src="./fig/9-3.png" width=700>
</p>

因此，类似于互相关性的定义，将这一项定义为交叉熵。同样地，通过交叉熵，我们也可以衡量两个分布间的差异性，若交叉熵较大，说明分布间差异较大。

In [60]:
# Cross Entropy Loss (Self-implemented)
def croEntLoss(predicted, ground_truth):
    loss = -ground_truth.matmul(predicted.log()).sum()
    return loss

print(croEntLoss(predicted_softmax, ground_truth))

# Torch Cross Entropy Loss
loss = nn.CrossEntropyLoss(reduction="sum")
CEL = loss(predicted_softmax.log(), ground_truth) # The input must be log(q)
print(CEL)

tensor(1.7328)
tensor(1.7328)
